In [ ]:
"""
Purpose: To implement the rest of the graph stat ideas that we had from the intro to networks book
"""

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Eigenvalues

In [ ]:
import scipy

def largest_adj_eigen_value(G1):
    Adj = nx.convert_matrix.to_numpy_matrix(G1)
    return np.real(np.max(np.linalg.eigvals(Adj)))

def smallest_adj_eigen_value(G1):
    Adj = nx.convert_matrix.to_numpy_matrix(G1)
    return np.real(np.min(np.linalg.eigvals(Adj)))

def largest_laplacian_eigen_value(G1):
    laplacian = scipy.sparse.csr_matrix.toarray(nx.laplacian_matrix(G1))
    return np.real(np.max(np.linalg.eigvals(laplacian)))

def smallest_laplacian_eigen_value(G1):
    laplacian = scipy.sparse.csr_matrix.toarray(nx.laplacian_matrix(G1))
    return np.real(np.min(np.linalg.eigvals(laplacian)))

def second_smallest_laplacian_eigen_value(G1):
    laplacian = scipy.sparse.csr_matrix.toarray(nx.laplacian_matrix(G1))
    sorted_eig_vals = np.sort(np.real(np.linalg.eigvals(laplacian)))
    return sorted_eig_vals[1]

#np.linalg.eigvals(np.array([[1,0],[0,1]]))
G1 = nx.erdos_renyi_graph(20,0.1)
print(largest_adj_eigen_value(G1))
print(smallest_adj_eigen_value(G1))
print(largest_laplacian_eigen_value(G1))
print(smallest_laplacian_eigen_value(G1))
print(second_smallest_laplacian_eigen_value(G1))

# Top heavy measurement

In [ ]:
from itertools import chain

def top_heavy_percentage(G1,top_percentage = 0.90):
    degree_sequence = np.array(G1.degree())[:,1]
    ordered_nodes = np.argsort(degree_sequence)


    index_to_start = np.ceil(len(degree_sequence)*top_percentage).astype("int")
    #print(f"index_to_start = {index_to_start}")
    top_nodes_to_keep = ordered_nodes[index_to_start:]
    #print(f"top_nodes_to_keep = {top_nodes_to_keep}")

    nodes_nbrs = G1.adj.items()
    top_neighbors = [set(v_nbrs) for v,v_nbrs in nodes_nbrs if v in top_nodes_to_keep]
    top_neighbors.append(set(top_nodes_to_keep))

    unique_top_neighbors = set(chain.from_iterable(top_neighbors))
    return len(unique_top_neighbors)/len(G1)

G1 = nx.erdos_renyi_graph(20,0.1)
top_heavy_percentage(G1)

# Critical Occupation Probability

In [ ]:
def critical_occupation_probability(G1):
    degree_sequence = np.array(G1.degree())[:,1]
    return np.mean(degree_sequence)/(np.mean(degree_sequence)**2 - np.mean(degree_sequence))

G1 = nx.erdos_renyi_graph(20,0.1)
critical_occupation_probability(G1)

# Triad closure on nodes of higher degree

In [ ]:
"""G = nx.erdos_renyi_graph(10,0.4)
degree_sequence = np.array(G.degree())[:,1]
unique_degrees = np.unique(degree_sequence)
same_or_higher_degree_env = dict([(k,np.where(degree_sequence>=k)[0]) for k in unique_degrees])"""

In [ ]:
def rich_club_transitivity(G):
    """
    Computes the triad closure percentage between only those nodes with same or higher degree
    """
    nodes_nbrs = G.adj.items()

    triads = 0
    triangles = 0
    degree_lookup = dict(G.degree())

    for v,v_nbrs in nodes_nbrs:
        v_nbrs_degree = [vnb for vnb in v_nbrs if degree_lookup[vnb] >= degree_lookup[v]]
        vs=set(v_nbrs_degree)-set([v]) #getting all the neighbors of the node (so when put in different combinations these could be triads)
        local_triangles=0
        local_triads = len(vs)*(len(vs) - 1)
        if local_triads<1:
            #print("No local triads so skipping")
            continue
        for w in vs:
            ws = set(G[w])-set([w]) #gets all the neighbors of a neighbor except itself
            local_triangles += len(vs.intersection(ws)) #finds out how many common neighbors has between itself and main node

        #print(f"For neuron {v}: Triads = {local_triads/2}, Triangles = {local_triangles/2}, transitivity = {local_triangles/local_triads}")
        triads += local_triads 
        triangles+= local_triangles
    
    #print(f"Total: Triads = {triads/2}, Triangles = {triangles/2}, transitivity = {triangles/triads}")
    if triads > 0:
        return triangles/triads
    else:
        return None
    
G = nx.erdos_renyi_graph(10,0.4)
rich_club_transitivity(G)

# Rich club coefficient

In [ ]:
G = nx.Graph([(0,1),(0,2),(1,2),(1,3),(1,4),(4,5)])
nx.draw(G,with_labels=True)
rc = nx.rich_club_coefficient(G,normalized=False)
rc

In [ ]:
"""
Not really sure what this means
"""

# Measure the modularity after performing clustering

# 1 / Average path length

In [ ]:
import networkx as nx


In [ ]:
def inverse_average_shortest_path(G):
    Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
    sp = nx.average_shortest_path_length(nx.subgraph(G,Gcc[0]))
    if sp > 0:
        return 1/sp
    else:
        return None
    
G = nx.erdos_renyi_graph(200,0.4)
inverse_average_shortest_path(G)

# Power Law Fitting Functions

In [ ]:
"""
Want to measure the goodness of fit of degree 
distribution between powerlaw and exponential

"""

In [4]:
import networkx as nx
import numpy as np
G = nx.erdos_renyi_graph(100,0.8)

In [9]:
import powerlaw

def get_degree_distribution(G):
    return np.array(G.degree())[:,1]

def power_law_alpha_sigma(G):
    #get the degree distribution
    power_law_alpha_sigma.stat_names = ["power_law_alpha",
                                        "power_law_sigma"]
    fit = powerlaw.Fit(get_degree_distribution(G))
    return fit.power_law.alpha, fit.power_law.sigma

def power_exp_fit_ratio(G):
    """
    Will return the loglikelihood ratio of the power and exponential graph
    R:
    Will be positive if power is more likely
            negative    exponential
    
    p: significance of fit
    """
    #get the degree distribution
    power_law_alpha_sigma.stat_names = ["power_exp_LL_ratio",
                                        "power_exp_LL_ratio_sign"]
    
    fit = powerlaw.Fit(get_degree_distribution(G))
    R,p = fit.distribution_compare("power_law",
                                                 "exponential",
                                                normalized_ratio=True)
    return R,p

def trunc_power_stretched_exp_fit_ratio(G):
    """
    Will return the loglikelihood ratio of the power and exponential graph
    R:
    Will be positive if power is more likely
            negative    exponential
    
    p: significance of fit
    """
    #get the degree distribution
    power_law_alpha_sigma.stat_names = ["trunc_power_stretched_exp_LL_ratio",
                                        "trunc_power_stretched_exp_LL_ratio_sign"]
    
    fit = powerlaw.Fit(get_degree_distribution(G))
    R,p = fit.distribution_compare("truncated_power_law",
                                                 "stretched_exponential",
                                                normalized_ratio=True)
    return R,pz
    

In [6]:
power_law_alpha_sigma(G)

Calculating best minimal value for power law fit


(46.463881143542956, 7.375215409225616)

In [13]:
power_exp_fit_ratio(G)

Calculating best minimal value for power law fit


(-0.3481321827483873, 0.7277409159108774)

In [14]:
trunc_power_stretched_exp_fit_ratio(G)

Calculating best minimal value for power law fit


(-0.14966877743063664, 0.8810259426615861)

In [8]:
dir(powerlaw
)

['Distribution',
 'Distribution_Fit',
 'Exponential',
 'Fit',
 'Lognormal',
 'Lognormal_Positive',
 'Power_Law',
 'Stretched_Exponential',
 'Truncated_Power_Law',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '__warningregistry__',
 'bisect_map',
 'ccdf',
 'cdf',
 'checkunique',
 'cumulative_distribution_function',
 'distribution_compare',
 'distribution_fit',
 'exponential_likelihoods',
 'find_xmin',
 'gamma_likelihoods',
 'is_discrete',
 'likelihood_function_generator',
 'loglikelihood_ratio',
 'lognormal_likelihoods',
 'negative_binomial_likelihoods',
 'nested_loglikelihood_ratio',
 'pdf',
 'plot_ccdf',
 'plot_cdf',
 'plot_pdf',
 'power_law_ks_distance',
 'power_law_likelihoods',
 'print_function',
 'stretched_exponential_likelihoods',
 'sys',
 'trim_to_range',
 'truncated_power_law_likelihoods']

In [ ]:
import powerlaw

fit = powerlaw.Fit(x)
print(fit.power_law.alpha, fit.power_law.sigma, fit.power_law.xmin)
print(fit.distribution_compare("power_law","exponential"))
f, ax = plt.subplots(figsize=(16,16))
ax.hist(x)
ax.set_yscale("log")
ax.set_xscale("log")

fit.power_law.plot_ccdf(ax = ax, color = "blue")
fit.plot_ccdf(ax = ax, color = "green")

In [ ]:
config_fit = powerlaw.Fit(total_degrees)
print(config_fit.power_law.alpha,config_fit.power_law.sigma,config_fit.distribution_compare("power_law","exponential"))